# **Machine Learning (Classificação) - Aprovação de Empréstimos**

### Dados da coluna:

 Este conjunto de dados fornece informações sobre as características dos requerentes de empréstimos e a sua avaliação de risco. Ele inclui informações sobre vários atributos dos solicitantes de empréstimo, incluindo detalhes demográficos, status financeiro, histórico de emprego e status de propriedade. O conjunto de dados inclui características numéricas e categóricas, tornando-o adequado para diversas abordagens analíticas.

 Principais características:

Id: Identificador exclusivo para cada solicitante de empréstimo.

Rendimento: O nível de rendimento do requerente.

Idade: Idade do candidato.

Experiência: Anos de experiência profissional.

Casado/Solteiro: Estado civil do requerente.

House_Ownership: Indica se o requerente possui ou aluga uma casa.

Car_Ownership: Indica se o requerente é proprietário de um carro.

Profissão: Ocupação ou profissão do candidato.

CIDADE: Cidade de residência do requerente.

ESTADO: Estado de residência do requerente.

CURRENT_JOB_YRS: Duração do emprego no emprego atual.

CURRENT_HOUSE_YRS: Duração da residência na casa atual.

Risk_Flag: Indicador binário de risco de empréstimo, onde 1 representa um candidato de risco sinalizado e 0 representa um candidato não arriscado.

## **Importando as Bibliotecas e Carregando o Dataset**

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,KFold,cross_val_score
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier


# ignorando avisos
warnings.filterwarnings('ignore')

# Estilo Dark nos gráficos do Matplotlib e Seaborn
plt.style.use('dark_background')

# Carregando os dados 
df = pd.read_json('/Users/diogomonteiro/loan_approval_dataset 3.json')

# Análisando os 5 primeiros dados
df.head(5)


KeyboardInterrupt



In [ ]:
# Analisando o tamanho dos dados, xxxxx linhas, xx colunas
df.shape

### **Análise Exploratória das Variáveis Categóricas e Numéricas'**

Nesta análise temos 2 objetivos:

1 - Conhecer as variáveis alvo (Risk_Flag) está relacionado com as outras variáveis.

2 - Iremos avaliar as variáveis categóricas para conhecimento dos dados e descartar variáveis que não fazem sentido.

3 - Conhecer as variáveis numéricas.

4 - Realizar uma análise estatística nas variáveis para o futuros tratamentos. Iremos avaliar média, mediana, moda, desvio padrão correlações, outliers, distruição dos dados, etc...


In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df2=df
df2['Risk_Flag'] = df['Risk_Flag'].astype(str)
df2.dtypes

In [ ]:
def analise(dados):
    for i in dados.columns:
        if dados[i].dtypes == 'object':
            print(f'\n Nome da coluna: {dados[i].name}\n')
            print(f'\n Tipo da coluna: {dados[i].dtypes}\n')
            print(f'\n Analisando os dados nulos: {dados[i].isnull().sum()}\n')
            plt.figure(figsize=(16,6))
            plt.title(dados[i].name)
            sns.countplot(data = dados, x = dados[i], hue = 'Risk_Flag')
            plt.show();

        else:
            print(f'\n Nome da coluna: {dados[i].name}\n')
            print(f'\n Tipo da coluna: {dados[i].dtypes}\n')
            print(f'\n Analisando os dados nulos: {dados[i].isnull().sum()}\n')
            print(f'\n Analisando os quartis:\n\n{dados[i].describe()}\n')
            plt.title(dados[i].name)
            sns.boxplot(data = dados[i])
            plt.show();
analise(df)

In [ ]:
df.groupby(['Profession']).size()

In [ ]:
df.groupby(['CITY']).size()

In [ ]:
df.groupby(['STATE']).size()

In [ ]:
df1= df.drop(['Id','Profession', 'STATE','CITY'],axis=1)

In [ ]:
df1.columns

In [ ]:
'''df1 = pd.get_dummies(df,columns=['House_Ownership', 'Profession', 'STATE'])
df1[['House_Ownership_norent_noown', 'House_Ownership_owned',
       'House_Ownership_rented', 'Profession_Air_traffic_controller',
       'Profession_Analyst', 'Profession_Architect', 'Profession_Army_officer',
       'Profession_Artist', 'Profession_Aviator',
       'Profession_Biomedical_Engineer', 'Profession_Chartered_Accountant',
       'Profession_Chef', 'Profession_Chemical_engineer',
       'Profession_Civil_engineer', 'Profession_Civil_servant',
       'Profession_Comedian', 'Profession_Computer_hardware_engineer',
       'Profession_Computer_operator', 'Profession_Consultant',
       'Profession_Dentist', 'Profession_Design_Engineer',
       'Profession_Designer', 'Profession_Drafter', 'Profession_Economist',
       'Profession_Engineer', 'Profession_Fashion_Designer',
       'Profession_Financial_Analyst', 'Profession_Firefighter',
       'Profession_Flight_attendant', 'Profession_Geologist',
       'Profession_Graphic_Designer', 'Profession_Hotel_Manager',
       'Profession_Industrial_Engineer', 'Profession_Lawyer',
       'Profession_Librarian', 'Profession_Magistrate',
       'Profession_Mechanical_engineer', 'Profession_Microbiologist',
       'Profession_Official', 'Profession_Petroleum_Engineer',
       'Profession_Physician', 'Profession_Police_officer',
       'Profession_Politician', 'Profession_Psychologist',
       'Profession_Scientist', 'Profession_Secretary',
       'Profession_Software_Developer', 'Profession_Statistician',
       'Profession_Surgeon', 'Profession_Surveyor',
       'Profession_Technical_writer', 'Profession_Technician',
       'Profession_Technology_specialist', 'Profession_Web_designer',
       'STATE_Andhra_Pradesh', 'STATE_Assam', 'STATE_Bihar',
       'STATE_Chandigarh', 'STATE_Chhattisgarh', 'STATE_Delhi',
       'STATE_Gujarat', 'STATE_Haryana', 'STATE_Himachal_Pradesh',
       'STATE_Jammu_and_Kashmir', 'STATE_Jharkhand', 'STATE_Karnataka',
       'STATE_Kerala', 'STATE_Madhya_Pradesh', 'STATE_Maharashtra',
       'STATE_Manipur', 'STATE_Mizoram', 'STATE_Odisha', 'STATE_Puducherry',
       'STATE_Punjab', 'STATE_Rajasthan', 'STATE_Sikkim', 'STATE_Tamil_Nadu',
       'STATE_Telangana', 'STATE_Tripura', 'STATE_Uttar_Pradesh',
       'STATE_Uttar_Pradesh[5]', 'STATE_Uttarakhand', 'STATE_West_Bengal']] = df1[['House_Ownership_norent_noown', 'House_Ownership_owned',
       'House_Ownership_rented', 'Profession_Air_traffic_controller',
       'Profession_Analyst', 'Profession_Architect', 'Profession_Army_officer',
       'Profession_Artist', 'Profession_Aviator',
       'Profession_Biomedical_Engineer', 'Profession_Chartered_Accountant',
       'Profession_Chef', 'Profession_Chemical_engineer',
       'Profession_Civil_engineer', 'Profession_Civil_servant',
       'Profession_Comedian', 'Profession_Computer_hardware_engineer',
       'Profession_Computer_operator', 'Profession_Consultant',
       'Profession_Dentist', 'Profession_Design_Engineer',
       'Profession_Designer', 'Profession_Drafter', 'Profession_Economist',
       'Profession_Engineer', 'Profession_Fashion_Designer',
       'Profession_Financial_Analyst', 'Profession_Firefighter',
       'Profession_Flight_attendant', 'Profession_Geologist',
       'Profession_Graphic_Designer', 'Profession_Hotel_Manager',
       'Profession_Industrial_Engineer', 'Profession_Lawyer',
       'Profession_Librarian', 'Profession_Magistrate',
       'Profession_Mechanical_engineer', 'Profession_Microbiologist',
       'Profession_Official', 'Profession_Petroleum_Engineer',
       'Profession_Physician', 'Profession_Police_officer',
       'Profession_Politician', 'Profession_Psychologist',
       'Profession_Scientist', 'Profession_Secretary',
       'Profession_Software_Developer', 'Profession_Statistician',
       'Profession_Surgeon', 'Profession_Surveyor',
       'Profession_Technical_writer', 'Profession_Technician',
       'Profession_Technology_specialist', 'Profession_Web_designer',
       'STATE_Andhra_Pradesh', 'STATE_Assam', 'STATE_Bihar',
       'STATE_Chandigarh', 'STATE_Chhattisgarh', 'STATE_Delhi',
       'STATE_Gujarat', 'STATE_Haryana', 'STATE_Himachal_Pradesh',
       'STATE_Jammu_and_Kashmir', 'STATE_Jharkhand', 'STATE_Karnataka',
       'STATE_Kerala', 'STATE_Madhya_Pradesh', 'STATE_Maharashtra',
       'STATE_Manipur', 'STATE_Mizoram', 'STATE_Odisha', 'STATE_Puducherry',
       'STATE_Punjab', 'STATE_Rajasthan', 'STATE_Sikkim', 'STATE_Tamil_Nadu',
       'STATE_Telangana', 'STATE_Tripura', 'STATE_Uttar_Pradesh',
       'STATE_Uttar_Pradesh[5]', 'STATE_Uttarakhand', 'STATE_West_Bengal']].astype('int64')'''
df1 = pd.get_dummies(df,columns=['House_Ownership'])
df1[['House_Ownership_norent_noown', 'House_Ownership_owned',
       'House_Ownership_rented']] = df1[['House_Ownership_norent_noown', 'House_Ownership_owned',
       'House_Ownership_rented']].astype('int64')
df1['Married/Single'].replace({'single': 0, 'married':1},inplace=True)
df1['Car_Ownership'].replace({'yes': 1, 'no': 0}, inplace=True)
df1

In [ ]:
df1 = df1.drop('CITY', axis=1)
df1

### **Resposta:**

Analisando os gráficos podemos ver que não temos outliers e não temos dados nulos, também vimos que a coluna City tem bastantes variáveis, por isso quando ultilizarmos get_dummies transformamos os dados categóricos em numéricos vamos ter muitas colunas.
Então vamos dividir os dados com a coluna City, e sem a coluna City, e vamos ver com o algoritimo se sai.

----

In [ ]:
x = df1.drop(['Profession','STATE','Risk_Flag'],axis=1)
x

In [ ]:
df3 = pd.get_dummies(df,columns=['Married/Single',
       'House_Ownership', 'Car_Ownership', 'Profession', 'CITY', 'STATE'])

In [ ]:
df3.columns

In [ ]:
df3 = df3.drop('Risk_Flag',axis=1)

In [ ]:
df3

In [ ]:
for i in df3.columns:
    if df3[i].dtypes == 'objetic':
        df3[i] = df3[i].astype('int64')
df3.dtypes        

In [ ]:
scaler1 = StandardScaler().fit_transform(df3)
scaler_x1 = pd.DataFrame(scaler1)
scaler_x1

In [ ]:
scaler = StandardScaler().fit_transform(x)
scaler_x = pd.DataFrame(scaler)
scaler_x

In [ ]:
y = df1.Risk_Flag.astype('int64')
y

In [ ]:
x_treino,x_teste,y_treino,y_teste = train_test_split(scaler_x,y, test_size = 0.3, random_state = 10)

In [ ]:
scaler_x.describe()

In [ ]:
modelos = {
    'naive_bayes': GaussianNB(),
    'logistic': LogisticRegression(tol=0.0001,C=1,solver='lbfgs',max_iter=600,random_state= 0,),
    'random': RandomForestClassifier(n_estimators=100,criterion='gini',random_state=0,max_depth=3),
    'arvore': DecisionTreeClassifier(criterion='gini',random_state=0,max_depth=3),
    'knn': KNeighborsClassifier(n_neighbors=7, metric='minkowski',p=5,),
    'xgboost': XGBClassifier()
}

class machine():

    def __init__(self,modelos,x_treino,x_teste,y_treino,y_teste):
        self.modelos = modelos
        self.x_treino = x_treino
        self.x_teste = x_teste
        self.y_treino = y_treino
        self.y_teste = y_teste
        self.modelos_score = []

    def relatorioML(self):

        
        
        for nome, model in self.modelos.items():

            
            # Ajustar o modelo com os dados de treino
            model.fit(self.x_treino, self.y_treino)
            
            # Previsões para o conjunto de treino e teste
            previsor_treino = model.predict(self.x_treino)
            previsor_teste = model.predict(self.x_teste)
            
            # Calcular a acurácia para treino e teste
            acuracia_treino = accuracy_score(self.y_treino, previsor_treino)
            acuracia_teste = accuracy_score(self.y_teste, previsor_teste)
            
            # Validação cruzada
            kf = KFold(n_splits=5, shuffle=True, random_state=5)
            
            resul = cross_val_score(model, self.x_treino, self.y_treino, cv=kf,n_jobs=-1)

            resul1 = cross_val_score(model, self.x_teste, self.y_teste, cv=kf,n_jobs=-1)

            resul_val_cruzada = round(resul.mean() * 100, 2)
            
            resul_val_cruzada1 = round(resul1.mean() * 100, 2)
            
            resultado_treino = round(model.score(self.x_treino,self.y_treino) * 100,2)
            resultado_teste = round(model.score(self.x_teste,self.y_teste) * 100,2)
        
            self.modelos_score.append({'Modelos': nome, 'Acurácia Treino':'{:.2f}%'.format(resultado_treino),
                                     'Acurácia Teste': '{:.2f}%'.format(resultado_teste),
                                    'Acurácia validacao cruzada treino': '{:.2f}%'.format(resul_val_cruzada),
                                      'Acurácia validacao cruzada teste': '{:.2f}%'.format(resul_val_cruzada1)})
    
        md = pd.DataFrame(self.modelos_score)

        return md

    def analiseML(self,modelo):
        
        previsor = modelo.predict(self.x_teste)
        
        previsor1 = modelo.predict(self.x_treino)


        print(f'\nTREINO')

        print('---------------------------------------------------------------------------')
        
        print('\n Acurácia treino: %.2f%%'%(accuracy_score(self.y_treino,previsor1)*100))

        print('---------------------------------------------------------------------------')

        print(f'\nConfusão matrix: \n{confusion_matrix(self.y_teste,previsor)}')

        print('---------------------------------------------------------------------------')
        
        print(f'\nClassificação: \n\n{classification_report(self.y_teste,previsor)}')

        print('---------------------------------------------------------------------------\n')

        print(f'\nTESTE')

        print('---------------------------------------------------------------------------')
        
        print('\n Acurácia teste: %.2f%%'%(accuracy_score(self.y_teste,previsor)*100))

        print('---------------------------------------------------------------------------')
        
        print(f'\nConfusão matrix: \n{confusion_matrix(self.y_teste,previsor)}')

        print('---------------------------------------------------------------------------')
        
        print(f'\nClassificação: \n\n{classification_report(self.y_teste,previsor)}')

        print('---------------------------------------------------------------------------')
        

    

Machine = machine(modelos,x_treino,x_teste,y_treino,y_teste)
Machine.relatorioML()

In [ ]:
Machine.analiseML(modelos['naive_bayes'])

In [ ]:
Machine.analiseML(modelos['logistic'])

In [ ]:
Machine.analiseML(modelos['arvore'])

In [ ]:
Machine.analiseML(modelos['random'])

In [ ]:
Machine.analiseML(modelos['knn'])

Primeiro teste

In [ ]:
Machine.analiseML(modelos['xgboost'])